In [ ]:
import warnings
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import zscore
from tableone import TableOne

In [ ]:
df = pd.read_csv('./path/to/case_control_file/csv')
df

In [ ]:
#filter out <18
df = df[df[('Patient_Age')]>18]
df

In [ ]:
#remove outliers
def removeExtremes(df, min_bilrubin):
    df_test = df.copy()
    q_low = df_test[min_bilrubin].quantile(0.001)
    q_hi  = df_test[min_bilrubin].quantile(0.95)
    df_test[min_bilrubin] = np.where((df_test[min_bilrubin] < q_hi) & (df_test[min_bilrubin] > q_low), df_test[min_bilrubin], np.nan)
    return df_test   
df = removeExtremes(df, 'min_bilrubin')
df['min_bilrubin'].describe()
###repeat for max_bilirubin

In [ ]:
df['mean_bilirubin'] = (df['max_bilrubin'] + df['min_bilrubin']) / 2
df.mean_bilirubin.describe()

In [ ]:
###smoking
formula = 'mean_bilirubin ~ smoke_hist + Sex + Patient_Age + SIRE_race + SIRE_eth'
m1 = smf.glm(formula = formula , data= df).fit()
m1.summary2()

In [ ]:
#restrict to current smokers
conditions = [
    (df['SmokingStatus'] == 'Every Day'),
    (df['SmokingStatus'] == 'Some Days'),
    (df['SmokingStatus'] == 'Light Smoker'),
    (df['SmokingStatus'] == 'Heavy Smoker'),
    (df['SmokingStatus'] == 'Never'),
    (df['SmokingStatus'] == 'Smoker, Current Status Unknown'),
    (df['SmokingStatus'] == 'Former'),
    (df['SmokingStatus'] == 'Passive Smoke Exposure - Never Smoker')
]

# Create a list of the values we want to assign for each condition
values = ['1', '1', '1', '1', '0', '1', np.nan, '0']

# Create a new column and use np.select to assign values to it using our lists as arguments
df['current_smoker'] = np.select(conditions, values)

# Display updated DataFrame
df

In [ ]:
###smoking
formula = 'mean_bilirubin ~ current_smoker + Sex + Patient_Age + SIRE_race + SIRE_eth'
m1 = smf.glm(formula = formula , data= df).fit()
m1.summary2()

In [ ]:
df['mean_bili_z'] = zscore(df['mean_bilirubin'])
df.head()

In [ ]:
#####hnc ####replace with LC for lung cancer analyses
formula = 'mean_bilirubin ~ HNC + Sex + Patient_Age + SIRE_race + SIRE_eth + smoke_hist'
m1 = smf.glm(formula = formula , data= df).fit()
m1.summary2()

In [ ]:
#regression models reverse direction
formula = 'HNC ~ zscore(mean_bilirubin) + Sex + Patient_Age + SIRE_race + SIRE_eth + smoke_hist'
m1 = smf.logit(formula = formula , data= df).fit()
m1.summary2()

In [ ]:
#regression models reverse direction
formula = 'HNC ~ mean_bilirubin + Sex + Patient_Age + SIRE_race + SIRE_eth + smoke_hist + mean_bili_z:smoke_hist'
m1 = smf.logit(formula = formula , data= df).fit()
m1.summary2()

In [ ]:
#####PGS######

In [ ]:
#input the PGS master file
pgs_master_file = pd.read_csv('./path/to/pgs/master/file/tsv',sep='\t')
pgs_master_file.head()

In [ ]:
bili_pgs = pgs_master_file[['ID','PGS002160']]
bili_pgs

In [ ]:
pheno_file_df = pd.read_csv('./path/to/genotyped/patients/file/csv')
pheno_file_df

In [ ]:
cc_df = pd.read_csv('./path/to/case_control_file/csv')
cc_df

In [ ]:
merge_df = pheno_file_df.merge(cc_df,on='ID')
merge_df

In [ ]:
final_df = merge_df.merge(bili_pgs,on='ID')
final_df

In [ ]:
final_df['mean_bilirubin'] = (final_df['max_bilirubin'] + final_df['min_bilirubin']) / 2
final_df.mean_bilirubin.describe()

In [ ]:
###restrict to European American Ancestry
ancestry = 'EUR'
pc_df = pd.read_csv(('./path/to/principal/component/file/EUR').format(ancestry), sep ='\t')
pc_df.head()

In [ ]:
eur_df = pc_df.merge(final_df,on='ID')
eur_df

In [ ]:
eur_df['pgs_2160_z'] =  (eur_df['PGS002160'] - eur_df['PGS002160'].mean()) / eur_df['PGS002160'].std()
eur_df.head()

In [ ]:
formula = 'mean_bilirubin ~ zscore(PGS002160) + Sex + Patient_Age + PC1 + PC2 + PC3 + PC4 + PC5'
m1 = smf.glm(formula = formula , data= eur_df).fit()
m1.summary2()

In [ ]:
#HNC-bili pgs
formula = 'HNC ~ zscore(PGS002160) + Sex + Patient_Age + PC1 + PC2 + PC3 + PC4 + PC5 + smoke_hist'
m1 = smf.logit(formula = formula , data= eur_df).fit()
m1.summary2()

In [ ]:
#lung cancer - bili pgs adjusted for smoking
formula = 'LC ~ zscore(PGS002160) + Sex + Patient_Age + PC1 + PC2 + PC3 + PC4 + PC5 + smoke_hist'
m1 = smf.logit(formula = formula , data= eur_df).fit()
m1.summary2()

In [ ]:
#####propensity score matching######

In [ ]:
data = pd.read_csv('./path/to/case_control_file/csv')
data

In [ ]:
#repeat with LC for lung cancer
formula = 'mean_bilirubin ~ HNC + Patient_Age + Sex + SIRE_eth + SIRE_race + smoke_hist'
m1 = smf.glm(formula = formula , data=data).fit()
m1.summary2()

In [ ]:
#propensity match towards HNC
# Define the columns to match on
matching_cols = ['Patient_Age', 'Sex', 'SIRE_eth', 'SIRE_race','smoke_hist']

# Define the treatment column
treatment_col = 'HNC'
#treatment_col = 'LC'

# One-hot encode the categorical columns
categorical_cols = ['Sex', 'SIRE_eth','SIRE_race','smoke_hist']
for col in categorical_cols:
    one_hot = pd.get_dummies(data[col], prefix=col)
    data = data.join(one_hot)

In [ ]:
# Filter the data to include only matched pairs
cases = data[data[treatment_col] == 1]
controls = data[data[treatment_col] == 0]

matched_controls = controls.groupby(matching_cols).apply(lambda x: x.sample(len(cases), replace=True)).reset_index(drop=True)
matched_data = pd.concat([cases, matched_controls], axis=0)

In [ ]:
X = pd.get_dummies(data[matching_cols], drop_first=True)

# Fit the logistic regression model to estimate the propensity scores
logit = sm.Logit(data[treatment_col], X)
propensity_scores = logit.fit().predict(X)

# Add the propensity scores to the original data
data['propensity_score'] = propensity_scores

In [ ]:
# Match cases and controls based on propensity scores
matched_data = pd.DataFrame(columns=data.columns)
for score in np.unique(propensity_scores):
    cases = data[(data[treatment_col] == 1) & (data['propensity_score'] == score)]
    controls = data[(data[treatment_col] == 0) & (data['propensity_score'] == score)]
    if len(cases) > 0 and len(controls) > 0:
        matched_controls = controls.sample(len(cases), replace=True)
        matched_data = pd.concat([matched_data, cases, matched_controls], axis=0)

In [ ]:
#repeat with LC for lung cancer
formula = ' mean_bilirubin ~ HNC'
m1 = smf.glm(formula = formula , data=matched_data).fit()
m1.summary2()